# **Training Using Multilayer Perceptron**


# **EEG Brain Age Estimation**  
## PCA + Data Augmentation + Deep MLP (Optuna) + BAG Analysis & Interpretability  
### Final Pipeline (2025)


This framework presents an end-to-end pipeline for **EEG-based Brain Age estimation**, combining signal preprocessing, dimensionality reduction, data augmentation, deep learning, and interpretability analysis.  
The goal is to predict **Brain Age** from EEG features and analyze the **Brain Age Gap (BAG)** as a biomarker of neural aging.

**Brain Age Gap (BAG)** is defined as:

\begin{equation}

\text{BAG} = \text{Predicted Brain Age} - \text{Chronological Age}

\end{equation}
A positive BAG may indicate accelerated brain aging, while a negative BAG may suggest preserved or delayed aging.

- EEG → PSD → Relative Band Power → Log Transform  
- PCA for dimensionality reduction  
- Data augmentation for robustness  
- Deep MLP optimized with Optuna  
- BAG analysis for neuroaging interpretation  

This framework provides a **robust, interpretable, and publication-ready approach** to EEG-based Brain Age estimation in 2025.


### **Librerias**

In [1]:
print("Installing packages...")

Installing packages...


In [2]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


Looking in indexes: https://download.pytorch.org/whl/cu118


In [1]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
import sys
!{sys.executable} -m pip uninstall -y numpy
!{sys.executable} -m pip install numpy==1.23.5


Found existing installation: numpy 1.23.5
Uninstalling numpy-1.23.5:
  Successfully uninstalled numpy-1.23.5


You can safely remove it manually.


  Using cached numpy-1.23.5-cp310-cp310-win_amd64.whl.metadata (2.3 kB)
Using cached numpy-1.23.5-cp310-cp310-win_amd64.whl (14.6 MB)


In [5]:
!pip install tensorflow==2.10.1 numpy pandas scipy matplotlib scikit-learn optuna protobuf==3.20.*


  Using cached protobuf-3.20.3-cp310-cp310-win_amd64.whl.metadata (698 bytes)
INFO: pip is looking at multiple versions of tensorflow to determine which version is compatible with other requirements. This could take a while.

The conflict is caused by:
    The user requested protobuf==3.20.*
    tensorflow 2.10.1 depends on protobuf<3.20 and >=3.9.2

Additionally, some packages in these conflicts have no matching distributions available for your environment:
    protobuf

To fix this you could try to:
1. loosen the range of package versions you've specified
2. remove package versions to allow pip to attempt to solve the dependency conflict



ERROR: Cannot install protobuf==3.20.* and tensorflow==2.10.1 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts


In [6]:
!pip install optuna

In [8]:
import sys
!{sys.executable} -m pip install tensorflow==2.10.1


  Using cached tensorflow-2.10.1-cp310-cp310-win_amd64.whl.metadata (3.1 kB)
  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.12.19-py2.py3-none-any.whl.metadata (1.0 kB)
  Using cached gast-0.4.0-py3-none-any.whl.metadata (1.1 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached h5py-3.15.1-cp310-cp310-win_amd64.whl.metadata (3.1 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl.metadata (1.9 kB)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached wheel-0.45.1-py3-none-any.whl.metadata (2.3 kB)
  Using cached idna-3.11-py3-none-any.whl.metadata (8.4 kB)
   ---------------------------------------- 0.0/455.9 MB ? eta -:--:--
   -----------------------

In [2]:
import os
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.inspection import permutation_importance
from tensorflow.keras import layers, models, callbacks, optimizers
import tensorflow as tf
import optuna
import warnings

c:\Users\Ale\Downloads\Jade\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
warnings.filterwarnings("ignore")

# Reproducibilidad
np.random.seed(42)
tf.keras.utils.set_random_seed(42)

### **Load EEG Dataset**

In [4]:
DATA_PATH = r"C:\Users\Ale\Downloads\Jade\Ale\Data_Parametrizada\EEG_features_final.csv"
SAVE_DIR = r"C:\Users\Ale\Downloads\Jade\Ale\Jade_Saves\Results_MLP"
os.makedirs(SAVE_DIR, exist_ok=True)

In [5]:
df = pd.read_csv(DATA_PATH)
df = df.select_dtypes(include=[np.number])
X = df.drop(columns=["Age"])
y = df["Age"]

print(f" Loaded {len(X)} samples, {X.shape[1]} features.")


 Loaded 510 samples, 9 features.


### **Cleaning and PCA**

In [6]:
z = np.abs(stats.zscore(X))
mask = (z < 3).all(axis=1)
X, y = X[mask], y[mask]

##Standardization
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

#PCA
pca = PCA(n_components=0.95, random_state=42)
X_pca = pca.fit_transform(X_scaled)
print(f" PCA reduced features to {X_pca.shape[1]} components.\n")

 PCA reduced features to 4 components.



### **Synthetic data augmentation**

In [7]:
X_aug, y_aug = X_pca.copy(), y.copy()
for _ in range(3):
    noise = np.random.normal(0, 0.01, X_pca.shape)
    X_aug = np.vstack([X_aug, X_pca + noise])
    y_aug = pd.concat([y_aug, y], ignore_index=True)

print(f" Augmented dataset: {X_aug.shape[0]} samples.\n")

print("Splitting data into train and test sets...")
X_train, X_test, y_train, y_test = train_test_split(X_aug, y_aug, test_size=0.2, random_state=42)
print(f" Training samples: {X_train.shape[0]}, Testing samples: {X_test.shape[0]}\n")


 Augmented dataset: 1968 samples.

Splitting data into train and test sets...
 Training samples: 1574, Testing samples: 394



## **MLP model for Optuna optimization**

### Funciones Auxiliares

In [8]:
def build_optuna_mlp(trial, input_dim):
    n_layers = trial.suggest_int("n_layers", 3, 7)
    lr = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)
    model = models.Sequential([layers.Input(shape=(input_dim,))])
    for i in range(n_layers):
        units = trial.suggest_int(f"units_{i}", 64, 256)
        drop = trial.suggest_float(f"drop_{i}", 0.1, 0.3)
        model.add(layers.Dense(units, activation="relu"))
        model.add(layers.Dropout(drop))
    model.add(layers.Dense(1, activation="linear"))
    model.compile(optimizer=optimizers.Adam(learning_rate=lr), loss="mae")
    return model

def objective(trial):
    model = build_optuna_mlp(trial, X_train.shape[1])
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 48])
    es = callbacks.EarlyStopping(monitor="val_loss", patience=20, restore_best_weights=True)
    rlrop = callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=10)
    model.fit(X_train, y_train, validation_split=0.2, epochs=250, batch_size=batch_size, verbose=0, callbacks=[es, rlrop])
    y_pred = model.predict(X_test, verbose=0).flatten()
    return r2_score(y_test, y_pred)

In [9]:
print(" Running Bayesian Optimization (Optuna + Deep MLP)...")
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=25, show_progress_bar=True)

print("\n[Optuna] Best R²:", f"{study.best_value:.3f}")
for k, v in study.best_params.items():
    print(f"  - {k}: {v}")

[I 2026-01-06 22:37:36,682] A new study created in memory with name: no-name-f2e656e2-85ed-4fe1-9a91-1d7bf1fbd754


 Running Bayesian Optimization (Optuna + Deep MLP)...


Best trial: 0. Best value: 0.924769:   4%|▍         | 1/25 [00:12<04:51, 12.15s/it]

[I 2026-01-06 22:37:48,900] Trial 0 finished with value: 0.9247687458992004 and parameters: {'n_layers': 3, 'learning_rate': 0.00882075265954828, 'units_0': 172, 'drop_0': 0.11790724657842884, 'units_1': 103, 'drop_1': 0.2626502283524199, 'units_2': 84, 'drop_2': 0.2764602260820379, 'batch_size': 48}. Best is trial 0 with value: 0.9247687458992004.


Best trial: 0. Best value: 0.924769:   8%|▊         | 2/25 [00:24<04:35, 11.98s/it]

...

[I 2026-01-06 22:52:03,275] Trial 22 finished with value: 0.9959330558776855 and parameters: {'n_layers': 6, 'learning_rate': 0.0017267037559065967, 'units_0': 256, 'drop_0': 0.2231905970874543, 'units_1': 256, 'drop_1': 0.12023741573779745, 'units_2': 121, 'drop_2': 0.17264328042103133, 'units_3': 241, 'drop_3': 0.16466655455455034, 'units_4': 145, 'drop_4': 0.1257211250457661, 'units_5': 87, 'drop_5': 0.2725523213263771, 'batch_size': 16}. Best is trial 11 with value: 0.9965032339096069.


Best trial: 11. Best value: 0.996503:  96%|█████████▌| 24/25 [15:12<00:51, 51.95s/it]

[I 2026-01-06 22:52:49,261] Trial 23 finished with value: 0.99506676197052 and parameters: {'n_layers': 5, 'learning_rate': 0.0009982069582759152, 'units_0': 227, 'drop_0': 0.21842607077575688, 'units_1': 211, 'drop_1': 0.12426996627257351, 'units_2': 153, 'drop_2': 0.16610212292334897, 'units_3': 232, 'drop_3': 0.1965977775015822, 'units_4': 135, 'drop_4': 0.12269187297351215, 'batch_size': 16}. Best is trial 11 with value: 0.9965032339096069.


Best trial: 11. Best value: 0.996503: 100%|██████████| 25/25 [15:40<00:00, 37.63s/it]

[I 2026-01-06 22:53:17,533] Trial 24 finished with value: 0.9329224824905396 and parameters: {'n_layers': 5, 'learning_rate': 0.0005927791200048803, 'units_0': 231, 'drop_0': 0.26045390369733284, 'units_1': 212, 'drop_1': 0.15290477003031616, 'units_2': 187, 'drop_2': 0.1626904653264714, 'units_3': 204, 'drop_3': 0.19573366274006987, 'units_4': 106, 'drop_4': 0.13161191836610256, 'batch_size': 16}. Best is trial 11 with value: 0.9965032339096069.

[Optuna] Best R²: 0.997
  - n_layers: 6
  - learning_rate: 0.001762115417413228
  - units_0: 220
  - drop_0: 0.23598318203682478
  - units_1: 253
  - drop_1: 0.10207106827993848
  - units_2: 112
  - drop_2: 0.16972254337668158
  - units_3: 252
  - drop_3: 0.15003806216599652
  - units_4: 151
  - drop_4: 0.10280432155912263
  - units_5: 123
  - drop_5: 0.2935165024711007
  - batch_size: 16


### **Final training with best parameters**

In [10]:
best_params = study.best_params
print("\nTraining final model with best hyperparameters...")
print(best_params)

final_model = build_optuna_mlp(study.best_trial, X_train.shape[1])
es = callbacks.EarlyStopping(monitor="val_loss", patience=25, restore_best_weights=True)
rlrop = callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=10)

final_model.fit(X_train, y_train, validation_split=0.2,
                epochs=400, batch_size=best_params["batch_size"],
                verbose=1, callbacks=[es, rlrop])

y_pred = final_model.predict(X_test, verbose=0).flatten()
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
r = np.corrcoef(y_test, y_pred)[0, 1]

print("\n================ FINAL TEST RESULTS =================")
print(f"MAE (years): {mae:.2f}")
print(f"R²: {r2:.3f}")
print(f"Pearson r: {r:.3f}")


Training final model with best hyperparameters...
{'n_layers': 6, 'learning_rate': 0.001762115417413228, 'units_0': 220, 'drop_0': 0.23598318203682478, 'units_1': 253, 'drop_1': 0.10207106827993848, 'units_2': 112, 'drop_2': 0.16972254337668158, 'units_3': 252, 'drop_3': 0.15003806216599652, 'units_4': 151, 'drop_4': 0.10280432155912263, 'units_5': 123, 'drop_5': 0.2935165024711007, 'batch_size': 16}
Epoch 1/400
79/79 [==============================] - 1s 5ms/step - loss: 25.9489 - val_loss: 19.0122 - lr: 0.0018
Epoch 2/400
79/79 [==============================] - 0s 4ms/step - loss: 19.1681 - val_loss: 17.8626 - lr: 0.0018
Epoch 3/400
79/79 [==============================] - 0s 4ms/step - loss: 18.7434 - val_loss: 17.3073 - lr: 0.0018
Epoch 4/400
79/79 [==============================] - 0s 4ms/step - loss: 17.6160 - val_loss: 16.4155 - lr: 0.0018
Epoch 5/400
79/79 [==============================] - 0s 4ms/step - loss: 16.4198 - val_loss: 16.7785 - lr: 0.0018
Epoch 6/400
79/79 [======

### **BAG Calculation and Categorization**

In [11]:
def categorize_bag(bag):
    if bag < -3: return "Resilient"
    elif bag > 3: return "Accelerated"
    else: return "Normal"

In [12]:
BAG = y_pred - y_test.values
bag_mean, bag_std = BAG.mean(), BAG.std()
print(f"\n Brain Age Gap (BAG): mean={bag_mean:.2f}, std={bag_std:.2f}")


bag_categories = np.vectorize(categorize_bag)(BAG)
bag_df = pd.DataFrame({
    "Chronological_Age": y_test.values,
    "Predicted_Age": y_pred,
    "BAG": BAG,
    "Category": bag_categories
})
bag_df.to_csv(os.path.join(SAVE_DIR, "EEG_Brain_Age_Gap_MLP.csv"), index=False)



 Brain Age Gap (BAG): mean=0.01, std=1.10


In [13]:
# Histogram
plt.figure(figsize=(7,5))
plt.hist(BAG, bins=25, color="skyblue", edgecolor="black")
plt.axvline(0, color="red", linestyle="--")
plt.title("Brain Age Gap (Predicted - Chronological)")
plt.xlabel("BAG (years)")
plt.ylabel("Count")
plt.grid(alpha=0.4)
plt.tight_layout()
plt.savefig(os.path.join(SAVE_DIR, "1.Bag_hist_mlp.png"))
plt.close()


In [14]:
# Category barplot
plt.figure(figsize=(6,4))
pd.Series(bag_categories).value_counts().reindex(["Resilient","Normal","Accelerated"]).plot(kind="bar", color=["#66c2a5","#fc8d62","#8da0cb"])
plt.title("BAG Categories Distribution (±3 years)")
plt.ylabel("Count")
plt.grid(axis="y", alpha=0.4)
plt.tight_layout()
plt.savefig(os.path.join(SAVE_DIR, "2.Bag_categories_mlp.png"))
plt.close()

### **Permutation Importance**

In [15]:
print("\n Computing permutation importance...")
X_test_df = pd.DataFrame(X_test, columns=[f"PC{i+1}" for i in range(X_test.shape[1])])
perm_result = permutation_importance(
    final_model, X_test_df, y_test, n_repeats=10, random_state=42, scoring="r2"
)
perm_df = pd.DataFrame({
    "Feature": X_test_df.columns,
    "Importance": perm_result.importances_mean
}).sort_values("Importance", ascending=False)

perm_df.to_csv(os.path.join(SAVE_DIR, "MLP_feature_importance.csv"), index=False)
plt.figure(figsize=(8,6))
plt.barh(perm_df["Feature"][:10], perm_df["Importance"][:10], color="cornflowerblue")
plt.gca().invert_yaxis()
plt.title("Top 10 Feature Importances (Permutation Importance – MLP)")
plt.xlabel("Mean R² Decrease")
plt.tight_layout()
plt.savefig(os.path.join(SAVE_DIR, "mlp_permutation_importance.png"))
plt.close()



 Computing permutation importance...
13/13 [==============================] - 0s 2ms/step


In [16]:
final_model.save(os.path.join(SAVE_DIR, "MLP_Deep_Optuna_FINAL.keras"))
pd.DataFrame([{
    "MAE": mae, "R2": r2, "r": r, "BAG_mean": bag_mean, "BAG_std": bag_std,
    "Best_Params": best_params, "PCA_Components": X_pca.shape[1]
}]).to_csv(os.path.join(SAVE_DIR, "MLP_Deep_Optuna_results.csv"), index=False)

print(f"\n All results saved in: {SAVE_DIR}")


 All results saved in: C:\Users\Ale\Downloads\Jade\Ale\Jade_Saves\Results_MLP


Due to the stochastic nature of multilayer perceptron training, the final model was trained independently 20 times using the same dataset and optimal hyperparameters. Performance metrics from each run were recorded to assess model robustness and stability, and results are reported as mean and standard deviation.

In [17]:
def BAG_stadistics(y_pred, y_test):
    BAG = y_pred - y_test.values
    bag_mean, bag_std = BAG.mean(), BAG.std()
    print(f"\n Brain Age Gap (BAG): mean={bag_mean:.2f}, std={bag_std:.2f}")
    return bag_mean, bag_std

    
def comparison_trials(n_trials, best_params, X_train, y_train, X_test, y_test):
    results = []
    for i in range(n_trials):
        print("\nTraining final model with best hyperparameters...")
        print("Trial:", i+1)
        final_model = build_optuna_mlp(study.best_trial, X_train.shape[1])
        es = callbacks.EarlyStopping(monitor="val_loss", patience=25, restore_best_weights=True)
        rlrop = callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=10)

        final_model.fit(X_train, y_train, validation_split=0.2,
                        epochs=400, batch_size=best_params["batch_size"],
                        verbose=1, callbacks=[es, rlrop])

        y_pred = final_model.predict(X_test, verbose=0).flatten()
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        r = np.corrcoef(y_test, y_pred)[0, 1]
        bag_mean, bag_std = BAG_stadistics(y_pred, y_test)

        results.append({
            "Trial": i + 1,
            "MAE": mae,
            "R2": r2,
            "Pearson_r": r,
            "BAG_mean": bag_mean,
            "BAG_std": bag_std,
            "Batch_size": best_params["batch_size"],
            "PCA_Components": X_pca.shape[1]
        })

    # save once at the end
    results_df = pd.DataFrame(results)
    results_df.to_csv(
        os.path.join(SAVE_DIR, "TRIALS_MLP_Deep_Optuna.csv"),
        index=False
    )

    print(f"\nAll results saved in: {SAVE_DIR}")



In [18]:
import time
star_time = time.time()
n=20
comparison_trials(20, best_params, X_train, y_train, X_test, y_test)
end_time = time.time()
print(f"\n Total comparison trials time: {end_time - star_time:.2f} seconds") 


Training final model with best hyperparameters...
Trial: 1
Epoch 1/400
79/79 [==============================] - 1s 4ms/step - loss: 24.8410 - val_loss: 18.8951 - lr: 0.0018
Epoch 2/400
79/79 [==============================] - 0s 3ms/step - loss: 19.1974 - val_loss: 18.1037 - lr: 0.0018
Epoch 3/400
79/79 [==============================] - 0s 3ms/step - loss: 17.9867 - val_loss: 16.6459 - lr: 0.0018
Epoch 4/400
79/79 [==============================] - 0s 3ms/step - loss: 16.9174 - val_loss: 16.3237 - lr: 0.0018
Epoch 5/400
79/79 [==============================] - 0s 3ms/step - loss: 16.0490 - val_loss: 16.7276 - lr: 0.0018
Epoch 6/400
79/79 [==============================] - 0s 3ms/step - loss: 14.9788 - val_loss: 13.4809 - lr: 0.0018
Epoch 7/400
79/79 [==============================] - 0s 3ms/step - loss: 15.3469 - val_loss: 14.4696 - lr: 0.0018
Epoch 8/400
79/79 [==============================] - 0s 3ms/step - loss: 14.1053 - val_loss: 12.5872 - lr: 0.0018
Epoch 9/400
79/79 [=========